# CoReference Network -- Richards

Notes
* Notebook currently treats the letter author and recipient as co-references. A strict author-recipient network at the moment (2021-09-27) would only have two nodes (Ellen Richards and Edward Atkinson).

In [1]:
import re, json, glob, csv, sys, os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import itertools as iter
import networkx as nx
from networkx.algorithms import community
from networkx.readwrite import json_graph
from json import JSONEncoder
from operator import itemgetter
from collections import Counter
    
# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/"

input_directory = "Data/PSC/Richards/ESR-XML-Files-MHS/*.xml"
output_file = "Data/Output/Graphs/Richards/Richards_coRef-network.json"

# Gather all .xml files using glob.
list_of_files = glob.glob(abs_dir + input_directory)

## Parse XML Files

In [2]:
%%time

'''
Arguments of Functions:

    namespace:

    ancestor:
    
    xpath_as_string:
    
    attrib_val_str:
    
'''

# Read in file and get root of XML tree.
def get_root(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root


# Get namespace of individual file from root element.
def get_namespace(root):
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    return ns

dataframe = []

for file in list_of_files:
    
    try:
        root = get_root(file)
        ns = get_namespace(root)

        reFile = str(re.search(r'.*/(.*.xml)', str(file)).group(1)) # get filename without path

        date = root.find('.//ns:date/[@type="creation"]', ns).get('when') # get date.

        source = root.find('.//ns:bibl//ns:author', ns).text   # get source/author & target/recipient
        target = root.find('.//ns:bibl//ns:recipient', ns).text
        
    #     Loops
    #     loop to get all references (persRef)
        references_l = []
        for ref in root.findall('.//ns:persRef', ns):
            person = ref.get('ref')
            references_l.append(person)
        references = ','.join(references_l)

    #     loop to get subjects.
        subject_l = []
        for subject in root.findall('.//ns:subject', ns):
            subject_l.append(subject.text)
        subjects = ','.join(subject_l)

    #     loop to get all text within <div type="docbody">
        text_l = []
        for txt in root.findall('.//ns:div[@type="docbody"]', ns):
            string = ''.join(ET.tostring(txt, encoding='unicode', method='text'))
            clean_string = re.sub(r'[\t\n\s]+', ' ', string)
            text_l.append(clean_string)
        content = ' '.join(text_l)


        row = {'file': reFile, 'date': date, 'source': source, 'target':target, 
               'subjects': subjects, 'references': references, 'text': content}
        

        dataframe.append(row)
        
    except:
        print (file, '\n')
    
dataframe = pd.DataFrame(dataframe)

dataframe.head(3)

/Users/quinn.wi/Documents/Data/PSC/Richards/ESR-XML-Files-MHS/ESR-EDA-1893-09-24.xml 

CPU times: user 9.83 ms, sys: 4.05 ms, total: 13.9 ms
Wall time: 15.9 ms


,file,date,source,target,subjects,references,text
0,ESR-EDA-1892-01-08.xml,1892-01-08,richards-ellen,atkinson-edward,"1893 Chicago World's Fair,Aladdin Oven,New Eng...","palmer-bertha,hovey-e,daniells-unknown",Boston Jan 8 1892 My dear Mr Atkinson I enclo...
1,ESR-EDA-1892-04-12.xml,1892-04-12,richards-ellen,atkinson-edward,"Aladdin Oven,nutrition,cooking",abel-mary,April 12— Dear Mr Atkinson I expect Mrs Abel ...
2,ESR-EDA-1892-04-07.xml,1892-04-07,richards-ellen,atkinson-edward,"Aladdin Oven,Nutrition,cooking","conro-emma,abel-mary","Boston, April 7, 1892 My dear Mr. Atkinson I ..."


## Reshape Dataframe for Network

In [3]:
%%time

# Split string of people into individuals.
dataframe['references'] = dataframe['references'].str.split(r',|;')

# Explode list so that each list value becomes a row.
refs = dataframe.explode('references')

# Create entry-person matrix.
refs = pd.crosstab(refs['file'], refs['references'])

# Repeat with correspondence (source + target)
source = pd.crosstab(dataframe['file'], dataframe['source'])
target = pd.crosstab(dataframe['file'], dataframe['target'])

# Join source & target to references as columns
refs = refs.join(source, on = 'file')
refs = refs.join(target, on = 'file')

# Convert entry-person matrix into an adjacency matrix of persons.
refs = refs.T.dot(refs)

# # Change diagonal values to zero. That is, a person cannot co-occur with themself.
# np.fill_diagonal(refs.values, 0)

# Create new 'source' column that corresponds to index (person).
refs['source'] = refs.index

# # Reshape dataframe to focus on source, target, and weight.
# # Rename 'people' column name to 'target'.
df = pd.melt(refs, id_vars = ['source'], var_name = 'target', value_name = 'weight') \
    .rename(columns = {'people':'target'}) \
    .query('(source != target) & (weight > 0)')

df

CPU times: user 47.5 ms, sys: 4.97 ms, total: 52.5 ms
Wall time: 57.6 ms


,source,target,weight
27,richards-ellen,,4
28,atkinson-edward,,4
31,abel-mary,abel-john,1
46,palmer-alice,abel-john,1
56,richards-ellen,abel-john,1
...,...,...,...
835,thompson-benjamin,atkinson-edward,1
836,unknown1,atkinson-edward,1
837,unknown2,atkinson-edward,1
838,unknown3,atkinson-edward,1


## Build Graph Object

In [4]:
%%time

# Initialize graph object.
G = nx.from_pandas_edgelist(df, 'source', 'target', 'weight')

# Add nodes.
nodes = list( dict.fromkeys( df['source'].values.tolist() + df['target'].values.tolist() ))
nodes = pd.DataFrame(nodes, columns = ['source'])
G.add_nodes_from(nodes)

print (nx.info(G))


# Set degree attributes.
nx.set_node_attributes(G, dict(G.degree(G.nodes())), 'degree')

# Sort nodes by degree and print top results.
sorted_degree = sorted(dict(G.degree(G.nodes())).items(),
                       key = itemgetter(1), reverse = True)

print ("Top 10 nodes by degree:")
for d in sorted_degree[:10]:
    print (f'\t{d[0]}, {d[1]}')

# Measure network density.
density = nx.density(G)
print (f"Network density: {density:.3f}")

# Related to diameter, check if network is connected and, therefore, can have a diameter.
print (f"Is the network connected? {nx.is_connected(G)}")

# Get a list of network components (communities).
# Find the largest component.
components = nx.connected_components(G)
largest_component = max(components, key = len)

# Create a subgraph of the largest component and measure its diameter.
subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print (f"Network diameter of the largest component: {diameter:.3f}")

# Find triadic closure (similar to density).
triadic_closure = nx.transitivity(G)
print (f"Triadic closure: {triadic_closure:.3f}\n")

# Find centrality measures.
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality
degree_cent_dict = nx.degree_centrality(G)

# Assign each centrality measure to an attribute.
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
nx.set_node_attributes(G, degree_cent_dict, 'degree_cent')

Name: 
Type: Graph
Number of nodes: 30
Number of edges: 98
Average degree:   6.5333
Top 10 nodes by degree:
	richards-ellen, 28
	atkinson-edward, 28
	daniells-unknown, 11
	abel-mary, 10
	hovey-e, 10
	davis-katherine, 7
	day-edna, 7
	unknown1, 7
	unknown2, 7
	unknown3, 7
Network density: 0.225
Is the network connected? False
Network diameter of the largest component: 2.000
Triadic closure: 0.393

CPU times: user 11 ms, sys: 1.19 ms, total: 12.2 ms
Wall time: 11.2 ms


## Write as Graph Object

In [5]:
%%time

# Convert graph object into a dictionary.
data = json_graph.node_link_data(G)

# Serialize dictionary with json.
class NPEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
    
data_json = json.dumps(data, cls=NPEncoder)

with open(abs_dir + output_file, "w") as f:
    f.write(data_json)

CPU times: user 2.42 ms, sys: 1.91 ms, total: 4.33 ms
Wall time: 3.84 ms
